# Classification multi classe

## Introduction aux données

+ Lire le fichier auto.csv dans un DataFrame nommé cars.
+ Utiliser la méthode Series.unique() pour assigner les éléments uniques de la colonne origin à la variable unique_regions. 
+ Afficher unique_regions.


In [2]:
import pandas as pd

cars = pd.read_csv('auto.csv')
print(cars.head())

    mpg  cylinders  displacement  horsepower  weight  acceleration  year  \
0  18.0          8         307.0       130.0  3504.0          12.0    70   
1  15.0          8         350.0       165.0  3693.0          11.5    70   
2  18.0          8         318.0       150.0  3436.0          11.0    70   
3  16.0          8         304.0       150.0  3433.0          12.0    70   
4  17.0          8         302.0       140.0  3449.0          10.5    70   

   origin  
0       1  
1       1  
2       1  
3       1  
4       1  


In [3]:
unique_regions = cars['origin'].unique()
print(unique_regions)

[1 3 2]


## Variables Nominales

+ Utiliser la fonction pandas.get_dummies() pour créer des valeurs factices de la colonne 'year'.
 - Utiliser le paramètret prefix fixé à year pour chacun des noms de colonne résultant.
 - Assigner la DataFrame résultant à dummy_years.
+ Utiliser la fonction pandas.concat() pour concaténer les colonnes du DataFrame dummy_years au DataFrame cars.
+ Utiliser la méthode DataFrame.drop() pour supprimer les colonnes year et cylinders du DataFrame cars. Assurez vous d'avoir repris le code précédent pour avoir les différentes colonnes cyl_ avant de supprimer la colonne cylinders.
+ Afficher les 5 premières lignes du nouveau DataFrame cars pour confirmer.


In [4]:
dummy_cylinders = pd.get_dummies(cars["cylinders"], prefix="cyl")
cars = pd.concat([cars, dummy_cylinders], axis=1)

dummy_years = pd.get_dummies(cars['year'], prefix='year')
cars = pd.concat([cars, dummy_years], axis=1)

cars = cars.drop('year', axis=1)
cars = cars.drop('cylinders', axis=1)

cars.head()

,mpg,displacement,horsepower,weight,acceleration,origin,cyl_3,cyl_4,cyl_5,cyl_6,...,year_73,year_74,year_75,year_76,year_77,year_78,year_79,year_80,year_81,year_82
0,18.0,307.0,130.0,3504.0,12.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,15.0,350.0,165.0,3693.0,11.5,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,18.0,318.0,150.0,3436.0,11.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,16.0,304.0,150.0,3433.0,12.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,17.0,302.0,140.0,3449.0,10.5,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Classification Multi-Classe

+ J'ai commencé par randomisé le DataFrame cars dans un DataFrame shuffled_cars pour vous.
+ Séparer le DataFrame shuffled_cars en 2 Dataframes: train et test.
 - Assigner les premiers 70% du DataFrame shuffled_cars au DataFrame train.
 - Assigner les 30% restants du DataFrame shuffled_cars au DataFrame test.


In [6]:
import numpy as np

shuffled_rows = np.random.permutation(cars.index)
shuffled_cars = cars.iloc[shuffled_rows]

highest_train_row = int(cars.shape[0]*.7)
train = shuffled_cars.iloc[0:highest_train_row]
test = shuffled_cars.iloc[highest_train_row:]

## Entrainer un modèle de régression logistique multi-classe

+ Utiliser la méthode Series.unique() pour assigner les éléments uniques de la colonne 'origin' à la variable unique_origins. Trier les 3 éléments de unique_origins.

Pour chaque valeur de unique_origins, entrainer un modèle de régression logistique avec les paramètres suivants:
+ X: Dataframe contenant uniquement les colonnes binaires 'cyl_' & 'year_'.
+ y: liste (ou objet Series) de valeurs booléennes :
 - True si la valeur de l'observation pour origin correspond à la variable itératrice de la boucle.
 - False si la valeur de l'observation pour origin ne correspond pas à la variable itératrice de la boucle.
 
Ajouter chaque modèle au dictionnaire models que vous aurez au préalable initialisé, ce dictionnaire aura la structure suivante:
+ Clé: valeur origine (1, 2 ou 3),
+ Valeur: instance pertinente du modèle LogisticRegression.


In [7]:
# str.startswith

In [8]:
from sklearn.linear_model import LogisticRegression

unique_origins = cars['origin'].unique()
unique_origins.sort()

models = {}
features = [c for c in train.columns if c.startswith("cyl") or c.startswith('year')]

for origin in unique_origins:
    model = LogisticRegression(solver='liblinear')
    
    X_train = train[features]
    y_train = train['origin'] == origin
    
    model.fit(X_train, y_train)
    models[origin] = model

## Tester les modèles

+ Soit le DataFrame testing_probs = pd.DataFrame(columns=unique_origins) qui possède 3 colonnes pour chacune des origines
+ Pour chaque valeur d'origine de unique_origins:
 - Utiliser la fonction predict_proba sur chacun des modèles pour retourner les 3 listes de probabilités prédites pour le set de test et l'ajouter au DataFrame testing_probs.
+ Voici à quoi devrait ressembler le DataFrame final:


| id  |  1  |  2  |  3  |
|---| --- | --- | --- |
| 0 | xxx | xxx | xxx |
| 1 | xxx | xxx | xxx |
| 2 | xxx | xxx | xxx |
| 3 | xxx | xxx | xxx |
| 4 | xxx | xxx | xxx |

In [9]:
testing_probs = pd.DataFrame(columns=unique_origins)

for origin in unique_origins:
    # Sélectionner les caractéristiques de test
    X_test = test[features]
    
    # Calculer la probabilité d'observation de la variable itératrice origin
    testing_probs[origin] = models[origin].predict_proba(X_test)[:,1]
    
testing_probs

,1,2,3
0,0.944979,0.036647,0.034808
1,0.238022,0.405930,0.355522
2,0.944979,0.036647,0.034808
3,0.665482,0.134109,0.223019
4,0.285587,0.295693,0.410682
5,0.885484,0.051983,0.085730
6,0.837471,0.048470,0.129563
7,0.875538,0.100285,0.042576
8,0.821307,0.060602,0.120020
9,0.238022,0.405930,0.355522


## Choisir l'origine de la voiture

+ Classifier chaque observation dans le set de test en utilisant le DataFrame testing_probs.
+ Assigner les origines prédites à la variable predicted_origins.
+ Afficher les 5 premières valeurs de predicted_origins et les comparer aux 5 premières lignes du DataFrame testing_probs.


In [10]:
# .idxmax(axis=1)

In [11]:
predicted_origins = testing_probs.idxmax(axis=1)

print(predicted_origins[0:5])
testing_probs.head()

0    1
1    2
2    1
3    1
4    3
dtype: int64


,1,2,3
0,0.944979,0.036647,0.034808
1,0.238022,0.405930,0.355522
2,0.944979,0.036647,0.034808
3,0.665482,0.134109,0.223019
4,0.285587,0.295693,0.410682
